In [ ]:
# =============================================================================
#  Imports
# =============================================================================

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'

from gs2mesh_utils.argument_utils import ArgParser
from gs2mesh_utils.colmap_utils import extract_frames, create_downsampled_colmap_dir, run_colmap, visualize_colmap_poses
from gs2mesh_utils.eval_utils import create_strings
from gs2mesh_utils.renderer_utils import Renderer
from gs2mesh_utils.stereo_utils import Stereo
from gs2mesh_utils.tsdf_utils import TSDF
from gs2mesh_utils.masker_utils import init_predictor, Masker

%load_ext autoreload
%autoreload 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
base_dir = os.path.abspath(os.getcwd())

**Parameters:** (edit only here)

In [ ]:
# =============================================================================
#  Initialize argument parser - DO NOT EDIT!
# =============================================================================
# Create argument parser with default arguments
args = ArgParser('custom')

# =============================================================================
#  Parameters - EDIT ONLY HERE!
# =============================================================================

# General params
args.colmap_name = 'sculpture' # Name of the directory with the COLMAP sparse model
args.dataset_name = 'custom' # Name of the dataset
args.experiment_folder_name = None # Name of the experiment folder

# Preprocessing params
args.downsample = 1 # Downsampling factor

# Gaussian Splatting parameters
args.GS_iterations = 30000  # Number of Gaussian Splatting iterations
args.GS_save_test_iterations = [7000, 30000]  # Gaussian Splatting test iterations to save
args.GS_white_background = False  # Use white background in Gaussian Splatting

# Renderer parameters
args.renderer_baseline_absolute = None  # Absolute value of the renderer baseline (None uses 7 percent of scene radius)
args.renderer_baseline_percentage = 7.0  # Percentage value of the renderer baseline
args.renderer_folder_name = None  # Name of the renderer folder (None uses the colmap name)
args.renderer_save_json = True  # Save renderer data to JSON
args.renderer_sort_cameras = False  # Sort cameras in the renderer (True if using unordered set of views)

# Stereo parameters
args.stereo_model = 'DLNR_Middlebury'  # Stereo model to use
args.stereo_occlusion_threshold = 3  # Occlusion threshold for stereo model (Lower value masks out more areas)
args.stereo_warm = True  # Use the previous disparity as initial disparity for current view (False if views are not sorted)

# TSDF parameters
args.TSDF_scale = 1.0  # Fix depth scale
args.TSDF_dilate = 1  # Take every n-th image (1 to take all images)
args.TSDF_valid = None  # Choose valid images as a list of indices (None to ignore)
args.TSDF_skip = None  # Choose non-valid images as a list of indices (None to ignore)
args.TSDF_use_occlusion_mask = True  # Ignore occluded regions in stereo pairs for better geometric consistency
args.TSDF_use_mask = False  # Use object masks (optional)
args.TSDF_invert_mask = False  # Invert the background mask for TSDF. Only if TSDF_use_mask is True
args.TSDF_erode_mask = True  # Erode masks in TSDF. Only if TSDF_use_mask is True
args.TSDF_erosion_kernel_size = 10  # Erosion kernel size in TSDF.  Only if TSDF_use_mask is True
args.TSDF_closing_kernel_size = 10  # Closing kernel size in TSDF.  Only if TSDF_use_mask is True.
args.TSDF_voxel = 2  # Voxel size (voxel length is TSDF_voxel/512)
args.TSDF_sdf_trunc = 0.04  # SDF truncation in TSDF
args.TSDF_min_depth_baselines = 4  # Minimum depth baselines in TSDF
args.TSDF_max_depth_baselines = 20  # Maximum depth baselines in TSDF
args.TSDF_cleaning_threshold = 100000  # Minimal cluster size for clean mesh

# Running parameters
args.video_extension = 'mp4'  # Video file extension
args.video_interval = 10  # Extract every n-th frame - aim for 3fps
args.GS_port = 8080  # GS port number (relevant if running several instances at the same time)
args.skip_video_extraction = False  # Skip the video extraction stage
args.skip_colmap = False  # Skip the COLMAP stage
args.skip_GS = False  # Skip the GS stage
args.skip_rendering = False  # Skip the rendering stage
args.skip_masking = False  # Skip the masking stage
args.skip_TSDF = False  # Skip the TSDF stage

# =============================================================================
#  DO NOT EDIT THESE LINES:
# =============================================================================
TSDF_voxel_length=args.TSDF_voxel/512
colmap_dir = os.path.abspath(os.path.join(base_dir,'data', args.dataset_name, args.colmap_name))
strings = create_strings(args)

**Extract frames if needed and Run COLMAP:** (only run if you don't have a COLMAP dataset. If you do, copy the colmap dataset to the "data" folder in the main root and update "colmap_output_dir")

In [ ]:
# =============================================================================
#  Extract frames from a video
# =============================================================================

if not args.skip_video_extraction:
    video_name = f'{args.colmap_name}.{args.video_extension}'
    extract_frames(os.path.join(colmap_dir, video_name), os.path.join(colmap_dir, 'images') , interval=args.video_interval)

In [ ]:
# =============================================================================
#  Create downsampled COLMAP directory
# =============================================================================

if args.downsample > 1:
    create_downsampled_colmap_dir(colmap_dir, args.downsample)
    args.colmap_name = f"{args.colmap_name}_downsample{args.downsample}"
    TSDF_voxel_length=args.TSDF_voxel/512
    colmap_dir = os.path.abspath(os.path.join(base_dir,'data', args.dataset_name, args.colmap_name))
    strings = create_strings(args)

In [ ]:
# =============================================================================
#  Run COLMAP with unknown poses
# =============================================================================

if not args.skip_colmap:
    run_colmap(colmap_dir, use_gpu=True) # If there's an error regarding SiftGPU not being supported, set use_gpu to False

In [ ]:
# =============================================================================
#  Visualize the sparse COLMAP output and the COLMAP poses.
# =============================================================================
GT_path = None # OPTIONAL: compare to a GT point cloud if it is aligned with the COLMAP sparse point cloud
visualize_colmap_poses(colmap_dir, depth_scale=10, subsample=100, visualize_points=True, GT_path=None) # if you don't see the cameras, adjust the depth scale. If you don't see the points, adjust the subsample

**Run Gaussian Splatting:**

In [ ]:
# =============================================================================
#  Run Gaussian Splatting
# =============================================================================

if not args.skip_GS:
    try:
        os.chdir(os.path.join(base_dir, 'third_party', 'gaussian-splatting'))
        iterations_str = ' '.join([str(iteration) for iteration in args.GS_save_test_iterations])
        os.system(f"python train.py -s {colmap_dir} --port {args.GS_port} --model_path {os.path.join(base_dir, 'splatting_output', strings['splatting'], args.colmap_name)} --iterations {args.GS_iterations} --test_iterations {iterations_str} --save_iterations {iterations_str}{' --white_background' if args.GS_white_background else ''}")
        os.chdir(base_dir)
    except:
        os.chdir(base_dir)
        print("ERROR")

**Prepare GS renderer for rendering stereo views:**

In [ ]:
# =============================================================================
#  Initialize renderer
# =============================================================================

renderer = Renderer(base_dir, 
                colmap_dir,
                strings['output_dir_root'],
                args.colmap_name, 
                dataset = strings['dataset'], 
                splatting = strings['splatting'],
                experiment_name = strings['experiment_name'],
                splatting_iteration = args.GS_iterations, 
                white_background = args.GS_white_background, 
                baseline_absolute = args.renderer_baseline_absolute, 
                baseline_percentage = float(args.renderer_baseline_percentage) * (2 if args.dataset_name=="DTU" else 1), 
                folder_name = args.renderer_folder_name,
                save_json = args.renderer_save_json,
                sort_cameras = args.renderer_sort_cameras,
                device=device)

In [ ]:
# =============================================================================
#  Visualize GS point cloud with COLMAP poses
# =============================================================================
renderer.visualize_poses(depth_scale=10, subsample=100)

In [ ]:
# =============================================================================
#  Prepare renderer
# =============================================================================
# ONLY NEED TO RUN ONCE PER SCENE!! Initializes renderer, takes some time
if not args.skip_rendering:
    renderer.prepare_renderer()

**Run Rendering + Stereo Model:**

In [ ]:
# =============================================================================
#  Initialize stereo
# =============================================================================

stereo = Stereo(base_dir, renderer, model_name=args.stereo_model, device=device)

In [ ]:
# =============================================================================
#  Run stereo
# =============================================================================
%matplotlib inline
#shading_eps relevant for visualization purposes. Change according to depth scale.
if not args.skip_rendering:
    stereo.run(shading_eps=1e-4, occlusion_threshold=args.stereo_occlusion_threshold, start=0, warm=args.stereo_warm, visualize=False)

**Run SAM2 Masker (OPTIONAL):**

In [ ]:
# =============================================================================
#  Initialize SAM2 predictor
# =============================================================================
# ONLY NEED TO RUN ONCE PER SCENE!! Initializes SAM2 predictor, takes some time
# Change use_local to True if you prefer using local weights of SAM2 instead of huggingface weights.
if not args.skip_masking:
    predictor, inference_state, images_dir = init_predictor(base_dir, renderer, use_local=False, device=device) 

In [ ]:
# =============================================================================
#  Find seed points for mask in reference image
# =============================================================================
# Run this cell at the beginning, and then run the next cell for automatic masking using SAM2.
# Left click: add positive point; Right click: add negative point; Left click + drag: add positive bounding box; Middle click - remove point/box (closest to click)
# IMPORTANT: bounding box only works when dragging from top left to bottom right!
# Zoom out for better handling if the image doesn't fit the screen.
%matplotlib widget
if not args.skip_masking:
    reference_image_num = 0
    masker = Masker(predictor, inference_state, images_dir, renderer, stereo, image_number=reference_image_num)

In [ ]:
# =============================================================================
#  Perform automatic masking using SAM2
# =============================================================================
%matplotlib inline
if not args.skip_masking:
    masker.segment()
    args.TSDF_use_mask = True

**View Results:**

In [ ]:
# ====================================================================================================
#  View left-right renders, segmentation mask, disparity, occlusion mask and shading (depth gradient)
# ====================================================================================================
%matplotlib inline
stereo.view_results()

**TSDF**

In [ ]:
# =============================================================================
#  Initialize TSDF
# =============================================================================

tsdf = TSDF(renderer, stereo, strings['TSDF'])

In [ ]:
# ================================================================================
#  Run TSDF. the TSDF class will have an attribute "mesh" with the resulting mesh
# ================================================================================
%matplotlib inline
if not args.skip_TSDF:
    tsdf.run(scale = args.TSDF_scale,
             dilate = args.TSDF_dilate,
             valid = args.TSDF_valid if args.TSDF_valid is not None else list(range(len(renderer))),
             skip = args.TSDF_skip if args.TSDF_skip is not None else [],
             use_occlusion_mask = args.TSDF_use_occlusion_mask, 
             use_mask = args.TSDF_use_mask, 
             invert_mask = args.TSDF_invert_mask,
             erode_mask = args.TSDF_erode_mask, 
             erosion_kernel_size = args.TSDF_erosion_kernel_size, 
             closing_kernel_size = args.TSDF_closing_kernel_size, 
             voxel_length = TSDF_voxel_length, 
             sdf_trunc = args.TSDF_sdf_trunc, 
             min_depth_baselines = args.TSDF_min_depth_baselines,
             max_depth_baselines = args.TSDF_max_depth_baselines, 
             visualize=False)

In [ ]:
# =============================================================================
#  Save the original mesh before cleaning
# =============================================================================

tsdf.save_mesh()

In [ ]:
# =============================================================================
#  Clean the mesh using clustering and save the cleaned mesh.
# =============================================================================

# original mesh is still available under tsdf.mesh (the cleaned is tsdf.clean_mesh)
tsdf.clean_mesh(thres=args.TSDF_cleaning_threshold/args.TSDF_scale)

In [ ]:
# =============================================================================
#  Show clean mesh
# =============================================================================
GT_path = None # OPTIONAL: compare to a GT point cloud if it is aligned with the COLMAP sparse point cloud
tsdf.visualize_mesh(subsample=100, GT_path=GT_path, show_clean=True)